# Using Dask and CuML with Google Cloud Dataproc

In this workshop, you will learn the use of Dask and CuML on Dataptoc.

__Dask__ is an open source library for parallel computing written in Python. Dask framework enables us to have a scheduler and a bunch of workers. You submit tasks to the scheduler and it automatically distributes the work among the workers. It works exceptionally well on a single machine, and can scale out to large clusters when needed. 
<img src="./images/dask-logo.png" width="200" height="200">

__RAPIDS CuML__ is a suite of fast, GPU-accelerated machine learning algorithms designed for data science and analytical tasks. Our API mirrors Sklearn’s, and we provide practitioners with the easy fit-predict-transform paradigm without ever having to program on a GPU.
<img src="./images/rapids_cuml.png" width="300" height="200">

__Dataproc__ is a fully managed and highly scalable service for running Apache Spark, Apache Flink, Presto, and 30+ open source tools and frameworks. It is a managed Spark and Hadoop service that lets you take advantage of open source data tools for batch processing, querying, streaming, and machine learning. Dataproc automation helps you create clusters quickly, manage them easily, and save money by turning clusters off when you don't need them. With less time and money spent on administration, you can focus on your jobs and your data. 
<img src="./images/gcp_dataproc_logo.png" width="400" height="300">


## Introduction

GPUs can greatly accelerate all stages of an ML pipeline: pre-processing, training, and inference. In this workshop, we will be focusing on the pre-processing and training stages, using Python in a Jupyter Notebook environment. First, we will use Dask/RAPIDS to read a dataset into NVIDIA GPU memory and execute some basic functions. Then, we’ll use Dask to scale beyond our GPU memory capacity.

This notebook has following sections:

* Introduction to Dataproc
* Introduction to Dask
* Data Loading
* ETL
* Introduction to CuML
* Introduction to XGBoost
* Machine Learning pipeline

## Introduction to Dask

Dask is the most commonly used parallelism framework within the PyData and SciPy communities. Dask is designed to scale from parallelizing workloads on the CPUs in your laptop to thousands of nodes in a cloud cluster. In conjunction with the open-source RAPIDS framework developed by NVIDIA, you can utilize the parallel processing power of both CPUs and NVIDIA GPUs. 

In Dask programming, we create computational graphs that define code we **would like** to execute, and then, give these computational graphs to a Dask scheduler which evaluates them lazily, and efficiently, in parallel.

In addition to using multiple CPU cores or threads to execute computational graphs in parallel, Dask schedulers can also be configured to execute computational graphs on multiple CPUs, or, as we will do in this workshop, multiple GPUs. As a result, Dask programming facilitates operating on datasets that are larger than the memory of a single compute resource.

In [ ]:

import certifi
import cudf
import cuml
import cupy as cp
import numpy as np
import os
import pandas as pd
import random
import seaborn as sns
import time
import yaml

from functools import partial
from math import cos, sin, asin, sqrt, pi
from tqdm import tqdm
from typing import Optional

# Dask imports
import dask
import dask.array as da
import dask_cudf

from dask_kubernetes import KubeCluster, make_pod_from_dict
from dask.distributed import WorkerPlugin, wait, progress


## Nvidia GPUs
Let us start by checking hardware available.

In [ ]:
!nvidia-smi -L

Other details like Memory-Usage and GPU utilization can be shown using following command.

In [ ]:
!nvidia-smi

### Starting a `LocalCUDACluster`

`dask_cuda` provides utilities for Dask and CUDA (the "cu" in cuDF) interactions.

In [ ]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()

## Instantiating a Client Connection
The `dask.distributed` library gives us distributed functionality, including the ability to connect to the CUDA Cluster we just created. The `progress` import will give us a handy progress bar we can utilize below.

In [ ]:
client = Client(cluster)
client

Dask library is smart enough to choose number of works and cores by default. In case of GPUs, `number of workers == number of GPUs` by default. In a distrubuted cluster environment, you can choose number of workers and cores. Dask ships with a very helpful dashboard that in our case runs on port `8787`.

In [ ]:
print('GPU Cluster status: ', client.status)

In [ ]:
# Query the client for all connected workers
# Number of workers in out cluster
workers = client.has_what().keys()
n_workers = len(workers)
print('Number of GPU workers: ', n_workers)


### Dataset

We are using [NYC Taxi Trip Duration Dataset from Kaggle](https://www.kaggle.com/c/nyc-taxi-trip-duration).

### Data fields

| Colonne            | Description                                                                                                                                                                                                           |
|:-------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| __id__                 | a unique identifier for each trip                                                                                                                                                                                     |
| __vendor_id__         | a code indicating the provider associated with the trip record                                                                                                                                                        |
| __pickup_datetime__    | date and time when the meter was engaged                                                                                                                                                                              |
| __dropoff_datetime__   | date and time when the meter was disengaged                                                                                                                                                                           |
| __passenger_count__    | the number of passengers in the vehicle (driver entered value)                                                                                                                                                        |
| __pickup_longitude__   | the longitude where the meter was engaged                                                                                                                                                                             |
| __pickup_latitude__    | the latitude where the meter was engaged                                                                                                                                                                              |
| __dropoff_longitude__  | the longitude where the meter was disengaged                                                                                                                                                                          |
| __dropoff_latitude__   | the latitude where the meter was disengaged                                                                                                                                                                           |
| __store_and_fwd_flag__ | This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server (Y=store and forward; N=not a store and forward trip) |
| __trip_duration__      | duration of the trip in seconds                                                                                                                                                                                       |

## ETL Exploration

### Taxi Data Configuration (Medium)
We can use the parquet data from the anaconda public repo here. Which will illustrate how much faster it is to read parquet, and gives us around 150 million rows of data to work with.

In [ ]:
def taxi_parquet_data_loader(client, response_dtype=np.float32, fraction=1.0, random_state=0):
    """
    A Parquet data_loader. Reads parquet files, clean and process to return a dataframe.
    
    Parameters:
    
    client: DASK client associated with the cluster we're interesting in collecting performance data for.
    fraction: Fraction of axis items to return. Cannot be used with n
    random_state: Seed for the random number generator (if int), or None. If None, a random seed will be chosen.
                if RandomState, seed will be extracted from current state.

    """
    # list of column names that need to be re-mapped
    remap = {}
    remap['tpep_pickup_datetime'] = 'pickup_datetime'
    remap['tpep_dropoff_datetime'] = 'dropoff_datetime'
    remap['ratecodeid'] = 'rate_code'

    #create a list of columns & dtypes the df must have
    must_haves = {
     'pickup_datetime': 'datetime64[ms]',
     'dropoff_datetime': 'datetime64[ms]',
     'passenger_count': 'int32',
     'trip_distance': 'float32',
     'pickup_longitude': 'float32',
     'pickup_latitude': 'float32',
     'rate_code': 'int32',
     'dropoff_longitude': 'float32',
     'dropoff_latitude': 'float32',
     'fare_amount': 'float32'
    }

    # apply a list of filter conditions to throw out records with missing or outlier values
    query_frags = [
        'fare_amount > 0 and fare_amount < 500',
        'passenger_count > 0 and passenger_count < 6',
        'pickup_longitude > -75 and pickup_longitude < -73',
        'dropoff_longitude > -75 and dropoff_longitude < -73',
        'pickup_latitude > 40 and pickup_latitude < 42',
        'dropoff_latitude > 40 and dropoff_latitude < 42'
    ]

    workers = client.has_what().keys()
    taxi_parquet_path = "gs://anaconda-public-data/nyc-taxi/nyc.parquet"
    response_id = 'fare_amount'
    fields = ['passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code',
                 'dropoff_longitude', 'dropoff_latitude', 'fare_amount']
    
    taxi_df = dask_cudf.read_parquet(taxi_parquet_path, npartitions=len(workers))
    taxi_df = clean(taxi_df, remap, must_haves)
    taxi_df = taxi_df.query(' and '.join(query_frags))
    taxi_df = taxi_df[fields]
    
    with dask.annotate(workers=set(workers)):
        taxi_df = client.persist(collections=taxi_df)
    
    wait(taxi_df)

    X = taxi_df[taxi_df.columns.difference([response_id])].astype(np.float32)
    y = taxi_df[response_id].astype(response_dtype)
        
    return taxi_df, X, y

def clean(df_part, remap, must_haves):
    """
    This function performs the various clean up tasks for the data
    and returns the cleaned dataframe.
    """
    tmp = {col:col.strip().lower() for col in list(df_part.columns)}
    df_part = df_part.rename(columns=tmp)
    
    # rename using the supplied mapping
    df_part = df_part.rename(columns=remap)
    
    # iterate through columns in this df partition
    for col in df_part.columns:
        # drop anything not in our expected list
        if col not in must_haves:
            df_part = df_part.drop(col, axis=1)
            continue

        # fixes datetime error found by Ty Mckercher and fixed by Paul Mahler
        if df_part[col].dtype == 'object' and col in ['pickup_datetime', 'dropoff_datetime']:
            df_part[col] = df_part[col].astype('datetime64[ms]')
            continue

        # if column was read as a string, recast as float
        if df_part[col].dtype == 'object':
            df_part[col] = df_part[col].astype('float32')
        else:
            # downcast from 64bit to 32bit types
            # Tesla T4 are faster on 32bit ops
            if 'int' in str(df_part[col].dtype):
                df_part[col] = df_part[col].astype('int32')
            if 'float' in str(df_part[col].dtype):
                df_part[col] = df_part[col].astype('float32')
            df_part[col] = df_part[col].fillna(-1)
            
    return df_part


In [ ]:
# list of column names that need to be re-mapped
remap = {}
remap['tpep_pickup_datetime'] = 'pickup_datetime'
remap['tpep_dropoff_datetime'] = 'dropoff_datetime'
remap['ratecodeid'] = 'rate_code'

#create a list of columns & dtypes the df must have
must_haves = {
 'pickup_datetime': 'datetime64[ms]',
 'dropoff_datetime': 'datetime64[ms]',
 'passenger_count': 'int32',
 'trip_distance': 'float32',
 'pickup_longitude': 'float32',
 'pickup_latitude': 'float32',
 'rate_code': 'int32',
 'dropoff_longitude': 'float32',
 'dropoff_latitude': 'float32',
 'fare_amount': 'float32'
}

# apply a list of filter conditions to throw out records with missing or outlier values
query_frags = [
    'fare_amount > 0 and fare_amount < 500',
    'passenger_count > 0 and passenger_count < 6',
    'pickup_longitude > -75 and pickup_longitude < -73',
    'dropoff_longitude > -75 and dropoff_longitude < -73',
    'pickup_latitude > 40 and pickup_latitude < 42',
    'dropoff_latitude > 40 and dropoff_latitude < 42'
]


In [ ]:
# This is path to our dataset and it is a parquet file. Can we load a CSV file too? Oh yes! 
taxi_parquet_path = "gs://anaconda-public-data/nyc-taxi/nyc.parquet"

# We are considering a limited columns for our ML pipeline.
response_id = 'fare_amount'
fields = ['passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code',
             'dropoff_longitude', 'dropoff_latitude', 'fare_amount']


### Load Dataset
By default, dask uses default block size of 64 MB each. So, `size of dataset / 64` will be number of tasks spread across partitions. If you desire to create a DataFrame with a specific block size, and use `npartitions` argument like below. We used `npartitions=len(workers)` to split data equally into all workers.

In [ ]:
%%time

taxi_df = dask_cudf.read_parquet(taxi_parquet_path, npartitions=len(workers))


In [ ]:
taxi_df = clean(taxi_df, remap, must_haves)
taxi_df = taxi_df.query(' and '.join(query_frags))
taxi_df = taxi_df[fields]


__Woah!__ I thought we will see data. What do you think has happened here?

Because Dask is lazy, the computation has not yet occurred. We can see that there are 160 tasks in the task graph. We can force computation by using `persist`. By forcing execution, the result is now explicitly in memory and our task graph only contains one task per partition (the baseline).

In [ ]:
response_dtype=np.float32

with dask.annotate(workers=set(workers)):
        taxi_df = client.persist(collections=taxi_df)
    
wait(taxi_df)

X = taxi_df[taxi_df.columns.difference([response_id])].astype(np.float32)
y = taxi_df[response_id].astype(response_dtype)


####  Split data to training and test set
We will use the `train_test_split()` method from `Dask_ML`.


In [ ]:
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, \
                                                    train_size=0.80, \
                                                    random_state=42, \
                                                    shuffle=True)


In [ ]:
X_train = X_train.persist()
X_test = X_test.persist()


## Ensemble Methods

Ensemble methods are learning algorithms that construct a set of classifiers and then classify new data points by taking a weighted vote of their predictions. There are three subsets of ensemble learning methods. 

1. __BAGGing__, or __B__ootstrap __AGG__regating
2. __Boosting__
3. __Stacking__

If you like to get more details about it, please refer to [Simple Guide to Ensemble Methods](https://towardsdatascience.com/simple-guide-for-ensemble-learning-methods-d87cc68705a2)


### Random Forest Regressor

The Decision Tree algorithm has a major disadvantage in that it causes over-fitting. To address these weaknesses, we turn to Random Forest. Random forest is a Supervised Learning algorithm which uses ensemble learning method for classification and regression. It is very fast and robust than other models. For anyone interested, we have added original papers and couple of blogs to learn more about Random Forest Regressor.

We are going to use [RandomForestRegressor API](https://docs.rapids.ai/api/cuml/stable/api.html#random-forest) from CuML library.



In [ ]:
from cuml.dask.ensemble import RandomForestRegressor

rf_kwargs = {
    "workers": client.has_what().keys(),
    "n_estimators": 10,
    "max_depth": 12
}
rf_csv_path = f"./{out_prefix}_random_forest_regression.csv"

performance_sweep(client=client, model=RandomForestRegressor,
                **sweep_kwargs,
                out_path=rf_csv_path,
                response_dtype=np.int32,
                model_kwargs=rf_kwargs)



In [ ]:
rf_csv_path = f"./{out_prefix}_random_forest_regression.csv"
visualize_csv_data(rf_csv_path)


### XGBoost

XGBoost falls under the category of Boosting techniques in Ensemble Learning. The algorithm was developed to efficiently reduce computing time and allocate an optimal usage of memory resources. Important features of implementation include handling of missing values (Sparse Aware), Block Structure to support parallelization in tree construction and the ability to fit and boost on new data added to a trained model. ([reference](https://www.kdnuggets.com/2017/10/xgboost-top-machine-learning-method-kaggle-explained.html)) 

Here is the original paper,
[XGBoost: A Scalable Tree Boosting System](https://arxiv.org/abs/1603.02754)


In [ ]:
import xgboost as xgb

dmatrix_train = xgb.dask.DaskDMatrix(client, X_train, y_train)
dmatrix_test = xgb.dask.DaskDMatrix(client, X_test, y_test)

params = {"max_depth": 8,
        "max_leaves": 2 ** 8,
        "alpha": 0.9,
        "eta": 0.1,
        "gamma": 0.1,
        "learning_rate": 0.1,
        "subsample": 1,
        "reg_lambda": 1,
        "scale_pos_weight": 2,
        "min_child_weight": 30,
        "tree_method": "gpu_hist", #use GPU for tree building
        "objective": "binary:logistic",
        "grow_policy": "lossguide",
    }



In [ ]:
%%time

# Use xgboost.dask to distribute across workers-- returns dict 
xgb_clasf = xgb.dask.train(client, 
                           params,
                           dmatrix_train, 
                           num_boost_round=100,
                           evals=[(dmatrix_train, 'train'), (dmatrix_test,'test')]
                          ) 

# booster is the trained model 
model_xgb = xgb_clasf['booster'] 


### Inferencing
#### Measuring model performance


In [ ]:
# Set to use GPU for inference.
model_xgb.set_param({'predictor': 'gpu_predictor'})
# dtrain is the DaskDMatrix defined above.
y_predict = xgb.dask.predict(client, xgb_clasf['booster'], dmatrix_test)
y_predict


In [ ]:
import dask 
y_test_c, y_predict = dask.compute(y_test, y_predict)
print("XGB model AUC: {:.2%}".format(cuml.metrics.roc_auc_score(y_test_c, y_predict)))


In [ ]:
print("XGBoost model accuracy:   {:.2%}".format(accuracy_score(y_test_c, y_predict)))

#### Resources

##### Dask

##### XGBoost
* [Ensemble Learning to Improve Machine Learning Results](https://blog.statsbot.co/ensemble-learning-d1dcd548e936)
* [Complete Guide to Parameter Tuning in XGBoost with codes in Python](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)
* [Understanding XGBoost Algorithm In Detail](https://analyticsindiamag.com/xgboost-internal-working-to-make-decision-trees-and-deduce-predictions/)

##### Random Forest Regressor
* [Random Forest](https://williamkoehrsen.medium.com/random-forest-simple-explanation-377895a60d2d) \
* [Random Forest Regression](https://towardsdatascience.com/machine-learning-basics-random-forest-regression-be3e1e3bb91a)
* [Classification and Regression by randomForest](https://www.researchgate.net/profile/Andy-Liaw/publication/228451484_Classification_and_Regression_by_RandomForest/links/53fb24cc0cf20a45497047ab/Classification-and-Regression-by-RandomForest.pdf)
